<a href="https://colab.research.google.com/github/realBarnaby/gpt_project/blob/main/chatgpt_beadando2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# letöltések
!pip install openai langchain serpapi google-search-results pypdf tiktoken faiss-cpu langchain_community gradio pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
# importok
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import AIMessage, HumanMessage

from google.colab import userdata

import openai
from openai import OpenAI

import os
import gdown
import gradio as gr

In [ ]:
# pdfek letöltése
url = "https://drive.google.com/drive/u/0/folders/1NRibpSwL0W9RRLhQTXWM0d-mr9jlFq2v"
gdown.download_folder(url)

Retrieving folder contents


Processing file 1J25SOIa1sAzz3bUlrpSi5kUyRuvT9IGf informatikai_biztonsag_alapveto_fogalmak.pdf
Processing file 1v4eGmNiaupNCahSTdwd-7fOE29L0IcPb informatikai_biztonsag_kiberhadviseles.pdf
Processing file 1O58r0DEwmO-ePoCplpvTiup_Et42zxKD informatikai_biztonsag_social_engineering.pdf
Processing file 1af9SYhvhk2XsIz6ZyRGzTCMVQ4RyBcKQ informatikai_biztonsag_teszteles_es_felugyelet.pdf
Processing file 1nIHoqOPGYPC-JSz_2rXWcjLoELRDgYrJ informatikai_biztonsag_titkositas_kulcsok.pdf
Processing file 1wd_D61QuwJesM-jNIzZVr6VALCmw1sYM informatikai_biztonsag_tuzfalak.pdf
Processing file 1PAJlO8tTqrr4smxAWPU3pjlVeEjztGBA informatikai_biztonsag_vezetek_nelkuli_halozatok_biztonsaga.pdf
Processing file 1r4KrIncCLWh3JJL5iiR6flLdnjxmu0nn szamitogep-halozatok_01_alapfogalmak.pdf
Processing file 1SZAXNCQUxFEcNKV78qX2VxnfBLcPvmL8 szamitogep-halozatok_02_retegszemlelet_osi_tcpip.pdf
Processing file 1nRcnkqOW72s1e9kv0-WczWokh7qm0M5Y szamitogep-halozatok_03_a_fizikai_reteg_es_eszkozei_atviteli_kozegek.pdf
Pr

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1J25SOIa1sAzz3bUlrpSi5kUyRuvT9IGf
To: /content/pdfek/informatikai_biztonsag_alapveto_fogalmak.pdf
100%|██████████| 105k/105k [00:00<00:00, 74.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v4eGmNiaupNCahSTdwd-7fOE29L0IcPb
To: /content/pdfek/informatikai_biztonsag_kiberhadviseles.pdf
100%|██████████| 186k/186k [00:00<00:00, 51.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O58r0DEwmO-ePoCplpvTiup_Et42zxKD
To: /content/pdfek/informatikai_biztonsag_social_engineering.pdf
100%|██████████| 90.4k/90.4k [00:00<00:00, 36.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1af9SYhvhk2XsIz6ZyRGzTCMVQ4RyBcKQ
To: /content/pdfek/informatikai_biztonsag_teszteles_es_felugyelet.pdf
100%|██████████| 174k/174k [00:00<00:00, 79.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nIHoqOPGYPC-JSz_2rXWcjLoELRDgYr

['/content/pdfek/informatikai_biztonsag_alapveto_fogalmak.pdf',
 '/content/pdfek/informatikai_biztonsag_kiberhadviseles.pdf',
 '/content/pdfek/informatikai_biztonsag_social_engineering.pdf',
 '/content/pdfek/informatikai_biztonsag_teszteles_es_felugyelet.pdf',
 '/content/pdfek/informatikai_biztonsag_titkositas_kulcsok.pdf',
 '/content/pdfek/informatikai_biztonsag_tuzfalak.pdf',
 '/content/pdfek/informatikai_biztonsag_vezetek_nelkuli_halozatok_biztonsaga.pdf',
 '/content/pdfek/szamitogep-halozatok_01_alapfogalmak.pdf',
 '/content/pdfek/szamitogep-halozatok_02_retegszemlelet_osi_tcpip.pdf',
 '/content/pdfek/szamitogep-halozatok_03_a_fizikai_reteg_es_eszkozei_atviteli_kozegek.pdf',
 '/content/pdfek/szamitogep-halozatok_04_adatkapcsolati_reteg.pdf',
 '/content/pdfek/szamitogep-halozatok_05_ethernet_alapismeretek.pdf',
 '/content/pdfek/szamitogep-halozatok_06_a_halozati_reteg-utvalasztas_torlodasok.pdf',
 '/content/pdfek/szamitogep-halozatok_07_a_halozati_reteg-qos_halozatok_osszekapcsolasa

In [13]:
# átírni az egészet mistral-os modellre
search = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm = ChatOpenAI(api_key = userdata.get('openai_api_key3'), temperature = 0)
llm_chain = LLMChain(llm=OpenAI(api_key = userdata.get('openai_api_key3')), prompt=prompt)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [ ]:
# a letöltött pdfek betöltése
path = "/content/pdfek/"
pdf_paths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.pdf')]

# kiiratás
pdf_paths

In [ ]:
# a pdfek feldarabolása
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,
            chunk_overlap=200
)

In [ ]:
# a pdfek betöltése egy tömbbe
pdfs = []
for pdf_path in pdf_paths:
  loader = PyMuPDFLoader(pdf_path)
  docs = loader.load()
  pdfs.extend(docs)

# kiiratás
pdfs

In [ ]:
# a pdfek eltárolása vektoradatbázisban
documents = text_splitter.split_documents(pdfs)
db = FAISS.from_documents(documents, OpenAIEmbeddings(api_key = userdata.get('openai_api_key3')))

In [ ]:
# ügynökök felvétele
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="keresésre való tool"
    ),

    Tool(
      name='Language Model',
      func=llm_chain.run,
      description='use this tool for general purpose queries and logic'
    ),

    create_retriever_tool(
        db.as_retriever(),
        "rebay_kereso",
        "Informatikai biztonság és számítógépes hálózatok",
)]

In [ ]:
# készítünk egy olyan ügynököt amely eldönti, hogy használjuk-e a vektoradatbázist
memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True
)

agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)
agent = initialize_agent(tools, llm = llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, language="hungarian")

In [ ]:
# ügynökök alkalmazása és használatukkal való keresés
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Mekkora kulccsal kódol a DES?'})

In [ ]:
qa_chain.run({'question':'Minek a rövidítése az RSA?'})

In [ ]:
qa_chain.run({'question':'Mondj példát, hogyan lehet bejutni védett helyekre'})

In [ ]:
agent.run({'page_content':None,'input':'Miért hívták a brit szigeteket Ködös Albionnak?'})

In [ ]:
qa_chain.run({'question':'Hogyan lehet osztályozni a tűzfalakat a megadott dokumentumok alapján?'})

In [ ]:
agent.run({'page_content':None,'input':'Hogyan lehet osztályozni a tűzfalakat a megadott dokumentumok alapján?'})

In [ ]:
agent.run({'page_content':None,'input':'Hogyan lehet osztályozni a tűzfalakat?'})

In [ ]:
agent.run({'page_content':None,'input':'Milyen a moduláris bővíthetősége a személyes tűzfalaknak a megadott dokumentumok alapján?'})

In [ ]:
# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent({'page_content':'None','input':'A csomagszűrő tűzfalak megvizsgálják az IP fejlécet?'})

In [ ]:
zero_shot_agent({'page_content':'None','input':'Milyen a moduláris bővíthetősége a személyes tűzfalaknak?'})

In [ ]:
zero_shot_agent({'page_content':'None','input':'Hogyan lehet osztályozni a tűzfalakat a megadott dokumentumok alapján?'})

In [ ]:
zero_shot_agent({'page_content':'None','input':'Hogyan lehet csoportosítani a tűzfalakat a megadott dokumentumok alapján?'})

In [ ]:
zero_shot_agent({'page_content':'None','input':'Hogyan lehet csoportosítani a tűzfalakat?'})

In [ ]:
qa_chain.run({'question':'Hogyan lehet osztályozni a tűzfalakat?'})

In [ ]:
# gradio
"""
llm = ChatOpenAI(api_key = userdata.get('openai_api_key3'), temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()
"""

def answer_question(question):
    response = agent.run({"page_content":None,"input":f"{question}"})
    return response

iface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
)
iface.launch(share=True)
